In [1]:
import cv2
import argparse
import numpy as np
import utils

In [2]:
# def detect_object(outs, list_images, Width, Height):

#     i = 0
#     dict_obj_detected = {}

#     # for each detetion from each output layer 
#     # get the confidence, class id, bounding box params
#     # and ignore weak detections (confidence < 0.5)
#     for out in outs:
#         for image in out:
#             image_name = list_images[i]
#             if not image_name in dict_obj_detected:
#                 dict_obj_detected[image_name] = {}
#                 dict_obj_detected[image_name]["class_ids"] = list()
#                 dict_obj_detected[image_name]["confidences"] = list()
#                 dict_obj_detected[image_name]["boxes"] = list()
#             for detection in image:
#                 scores = detection[5:]
#                 class_id = np.argmax(scores)
#                 confidence = scores[class_id]
#                 if confidence > 0.5:
#                     center_x = int(detection[0] * Width[i])
#                     center_y = int(detection[1] * Height[i])
#                     w = int(detection[2] * Width[i])
#                     h = int(detection[3] * Height[i])
#                     x = center_x - w / 2
#                     y = center_y - h / 2
#                     dict_obj_detected[image_name]["class_ids"].append(class_id)
#                     dict_obj_detected[image_name]["confidences"].append(float(confidence))
#                     dict_obj_detected[image_name]["boxes"].append([x, y, w, h])
#             i += 1
#         i = 0 

#     return dict_obj_detected

# def detect_danger(dict_obj_detected, idx_class=0):
    
#     dict_image_danger = {}
    
#     for image_name, row in dict_obj_detected.items():
#         if len(row["class_ids"]) != 0 and idx_class in row["class_ids"]: 
#             dict_image_danger[image_name] = True
#         else:
#             dict_image_danger[image_name] = False
    
#     return dict_image_danger

In [3]:
%%time
# read input image
list_images = utils.get_files_by_ext("object-detection-opencv/", "jpg")
classes_file = "object-detection-opencv/yolov3.txt"
weights_file = "object-detection-opencv/yolov3.weights"
conf_file = "object-detection-opencv/yolov3.cfg"

conf_threshold = 0.5
nms_threshold = 0.4

Width = list()
Height = list()
for image_path in list_images:
    image = cv2.imread(image_path)

    Width.append(image.shape[1])
    Height.append(image.shape[0])

scale = 0.00392

# read class names from text file
classes = None
with open(classes_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# create input blob 
# prepares the input image to run through the deep neural network
blob = utils.image_to_matrix(list_images)

# read pre-trained model and config file
# reads the weights and config file and creates the network.
net = cv2.dnn.readNet(weights_file, conf_file)

# set input blob for the network
net.setInput(blob)

# run inference through the network
# and gather predictions from output layers
outs = net.forward(utils.get_output_layers(net))

# identify objects into the list of images analyzed by the network
dict_obj_detected = {}
dict_obj_detected = utils.detect_object(outs, list_images, Width, Height)

# identify pictures with at least one perso
dict_danger = utils.detect_danger(dict_obj_detected)
for image_name, is_dangerous in dict_danger.items():
    print("Is", image_name, "dangerous ? \t\t\t" , is_dangerous)

# Draw bounding boxes and save images
for image_path, image_items in dict_obj_detected.items():
    utils.get_bounding_box(image_path, image_items, classes, COLORS, conf_threshold, nms_threshold)

Is object-detection-opencv/344.jpg dangerous ? 			 True
Is object-detection-opencv/dog.jpg dangerous ? 			 False
Is object-detection-opencv/object-detection.jpg dangerous ? 			 False
Is object-detection-opencv/test.jpg dangerous ? 			 True
Is object-detection-opencv/test_no_person.jpg dangerous ? 			 False
CPU times: user 12.6 s, sys: 373 ms, total: 12.9 s
Wall time: 8.42 s


## Note : ToDo
The cells below are commented because they still needs to be updated

In [4]:
# def init_detect_person():
#     # initialization
#     conf_threshold = 0.5
#     nms_threshold = 0.4
    
#     return conf_threshold, nms_threshold

In [5]:

# def get_bounding_box(image_path, image_items, classes, conf_threshold, nms_threshold):
# #   apply non-max suppression
#     indices = cv2.dnn.NMSBoxes(image_items["boxes"], image_items["confidences"], conf_threshold, nms_threshold)
#     image = cv2.imread(image_path)

# #   go through the detections remaining
# #   after nms and draw bounding box
#     for i in indices:
#         i = i[0]
#         box = image_items["boxes"][i]
#         x = box[0]
#         y = box[1]
#         w = box[2]
#         h = box[3]

#         draw_bounding_box(image, image_items["class_ids"][i], image_items["confidences"][i], \
#                                     round(x), round(y), round(x+w), round(y+h), \
#                                     classes
#                                )
#     # save output image to disk
#     cv2.imwrite("output_with_bounding_box/" + image_path, image)

#     # release resources
#     # cv2.destroyAllWindows()



In [6]:
# # function to draw bounding box on the detected object with class name
# def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h, classes):

#     label = str(classes[class_id])

#     color = COLORS[class_id]

#     cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

#     cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)